In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

import warnings
warnings.filterwarnings('ignore')

from PIL import Image
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import *

In [ ]:
data = []
labels = []
classes = 43
cur_path = '../input/gtsrb-german-traffic-sign/Train'

for i in os.listdir(cur_path):
    dir = cur_path + '/' + i
    for j in os.listdir(dir):
        img_path = dir+'/'+j
        img = cv2.imread(img_path,-1)
        img = cv2.resize(img, (30,30), interpolation = cv2.INTER_NEAREST)
        data.append(img)
        labels.append(i)
        
x_train = np.array(data)
x_train = x_train / 255
y_train = np.array(labels).astype(np.float)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)

In [ ]:
np.unique(labels).shape

In [ ]:
i_path = '../input/gtsrb-german-traffic-sign/Train/1/00001_00072_00027.png'
plt.imshow(cv2.imread(i_path, -1))

In [ ]:
data_dic = {}
for folder in os.listdir(cur_path):
    data_dic[folder] = len(os.listdir(cur_path + '/' + folder))

data_df= pd.Series(data_dic)
plt.figure(figsize = (15, 6))
data_df.sort_values().plot(kind = 'bar')
plt.xlabel('Classes')
plt.ylabel('Number of images')

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (5,5), activation = 'relu', 
                 input_shape = (30, 30, 3), data_format = 'channels_last',
                 kernel_regularizer = regularizers.l1_l2(l1=1e-4, l2=1e-3)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = "same", strides = (2, 2),
                kernel_regularizer = regularizers.l1_l2(l1=1e-4, l2=1e-3)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 128, kernel_size = (5,5), activation = 'relu' ,
                 kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(filters = 128, kernel_size = (5,5), padding = "same",
                 kernel_regularizer = regularizers.l1_l2(l1=1e-4, l2=1e-3), strides = (2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(1024, activation = 'relu',  kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3)))
model.add(Dropout(0.4))
model.add(Dense(256, activation = 'relu',  kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3)))
model.add(Dropout(0.5))
model.add(Dense(43, activation = 'softmax',  kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3)))
model.summary()


In [ ]:
callback = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',factor=1e-1, patience=8, verbose=1, min_lr = 2e-6)
opt = Adam(learning_rate = 1e-3)
model.compile(optimizer = opt,
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train ,y_train,  batch_size = 128, 
         epochs=200, validation_data = (x_valid, y_valid),
          callbacks=[callback, reduce_lr], shuffle = True)

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label= 'train accuracy')
plt.plot(history.history['val_accuracy'], label= 'test accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
plt.figure(0)
plt.plot(history.history['loss'], label= 'train loss')
plt.plot(history.history['val_loss'], label= 'test loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
test = pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv')
y_test = test["ClassId"].values
imgs = test["Path"].values
data =[]
for img in imgs:
    image = cv2.imread('../input/gtsrb-german-traffic-sign/' +img)
    image_fromarray = Image.fromarray(image, 'RGB')
    resize_image = image_fromarray.resize((30, 30))
    data.append(np.array( resize_image ))

x_test = np.array(data)
x_test = x_test/255
model.evaluate(x_test, y_test)

In [ ]:
dic = {0 : '20 speed limit', 1 : '30 speed limit', 2 : '50 speed limit', 3 : '60 speed limit', 4 : '70 speed limit', 5 : '80 speed limit', 6 : 'do not cross 80', 7 : '100 speed limit', 8 : '120 speed limit', 9 : 'red car and black car opposite sign board traffic', 10 : 'red truck and black car opposite sign board traffic', 11 : 'priority road with junction', 12 : 'priority road', 13 : 'Yeild ahead', 14 : 'stop', 15 : 'Obligatory', 16 : 'heavy load trucks', 17 : 'No entry', 18 : 'warning', 19 : 'take left', 20 : 'take right', 21 : 'zig zag road', 22 : 'speed breaker', 23 : 'slippery when wet', 24 : 'lane reduction', 25 : 'work in progress', 26 : 'traffic signals ahead', 27 : 'pedestrain crossing ahead', 28 : 'patrol', 29 : 'cycle route ahead', 30 : 'snowflake', 31 : 'deer crossing', 32: 'no symbol', 33 : 'take right', 34 : 'take left', 35 : 'go straight', 36 : 'Right Turn Or Straight', 37 : 'Left Turn Or Straight', 38 : 'keep right', 39 : 'keep left', 40 : 'Mini-roundabout', 41 : 'no passing two cars', 42 : 'no passing two cars', 43 : 'no passing truck and car'} 

In [ ]:
def predictImage(image_path):
    image = cv2.imread(image_path, -1)
    plt.imshow(image)
    plt.show()
    image = cv2.resize(image, (30,30), interpolation = cv2.INTER_NEAREST)
    image = image.reshape((1,30,30,3))
    prediction = model.predict(image)[0]
    predicted_class = np.argmax(prediction)
    pred_prob = max(prediction)
    return dic[predicted_class]

In [ ]:
predictImage("../input/gtsrb-german-traffic-sign/Test/00390.png")

In [ ]:
model.save('model.h5')

In [ ]:
import tensorflow
tensorflow.__version__

In [ ]:
!python --version